In [4]:
from DatasetHandler.BiwiBrowser import *
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [2]:
subjectList = [1, 2, 3, 4, 5, 7, 8, 11, 12, 14] # except [6, 10, 13, ]
testSubjects = [9]

In [3]:
biwi = readBIWIDataset(subjectList = subjectList, scaling = False)

All frames and annotations from 10 datasets have been read by 2018-12-30 14:43:31.699601


In [6]:
images = []
for inputMatrix, labels in biwi:
    images.append(inputMatrix[40])

In [8]:
images[0].shape

(3, 480, 640)

In [12]:
def rolling_window(m):
    shape = m.shape[1:] + (m.shape[0],)
    strides = m.strides + (m.strides[0],)
    return numpy.lib.stride_tricks.as_strided(m, shape=shape, strides=strides)


In [13]:
plt.imshow(rolling_window(images[0]))

ValueError: mismatch in length of strides and shape